# Day 5: Hydrothermal Venture

You come across a field of [hydrothermal vents](https://en.wikipedia.org/wiki/Hydrothermal_vent) on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in **lines**; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```

Each line of vents is given as a line segment in the format `x1,y1 -> x2,y2` where `x1`,`y1` are the coordinates of one end the line segment and `x2`,`y2` are the coordinates of the other end. These line segments include the points at both ends. In other words:

- An entry like `1,1 -> 1,3` covers points `1,1`, `1,2`, and `1,3`.
- An entry like `9,7 -> 7,7` covers points `9,7`, `8,7`, and `7,7`.

## Part 1

For now, **only consider horizontal and vertical lines**: lines where either `x1 = x2` or `y1 = y2`.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```

In this diagram, the top left corner is `0,0` and the bottom right corner is `9,9`. Each position is shown as **the number of lines which cover that point** or `.` if no line covers that point. The top-left pair of `1`s, for example, comes from `2,2 -> 2,1`; the very bottom row is formed by the overlapping lines `0,9 -> 5,9` and `0,9 -> 2,9`.

To avoid the most dangerous areas, you need to determine **the number of points where at least two lines overlap**. In the above example, this is anywhere in the diagram with a `2` or larger - a total of `5` points.

Consider only horizontal and vertical lines. **At how many points do at least two lines overlap?**


In [1]:
from numpy import sign 


if __name__ == '__main__':
    vent_lines = []
    vent_map = dict()

    # read input file
    with open('./input.txt') as input_file:
        for line in [line.strip() for line in input_file.readlines()]:
            vent_lines.append([int(n) for pair in line.split(' -> ') for n in pair.split(',')])

    # remove non horizontal or verticel lines
    vent_lines = [line for line in vent_lines if line[0] == line[2] or line[1] == line[3]]

    # fill map
    for line in vent_lines:
        # vertical line
        if (line[0] == line[2]):
            direction = sign(line[3] - line[1])
            for y in range(line[1], line[3] + direction, direction):
                if (line[0], y) not in vent_map:
                    vent_map[(line[0], y)] = 1
                else:
                    vent_map[(line[0], y)] += 1

        # horizontal line
        elif (line[1] == line[3]):
            direction = sign(line[2] - line[0])
            for x in range(line[0], line[2] + direction, direction):
                if (x, line[1]) not in vent_map:
                    vent_map[(x, line[1])] = 1
                else:
                    vent_map[(x, line[1])] += 1

        # Shouldn't hapen in this part of the challenge.
        else:
            print("ERROR: Line not horizontal or vertical")

    # count dangerous coords
    count = len([coord for coord in vent_map if vent_map[coord] > 1])

    print(f'Solution: {count}')

Solution: 4745


## Part 2

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider **diagonal lines**.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

- An entry like `1,1 -> 3,3` covers points `1,1`, `2,2`, and `3,3`.
- An entry like `9,7 -> 7,9` covers points `9,7`, `8,8`, and `7,9`.

Considering all lines from the above example would now produce the following diagram:

```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine **the number of points where at least two lines overlap**. In the above example, this is still anywhere in the diagram with a `2` or larger - now a total of `12` points.

Consider all of the lines. **At how many points do at least two lines overlap?**

In [2]:
from numpy import sign 


if __name__ == '__main__':
    vent_lines = []
    vent_map = dict()

    # read input file
    with open('./input.txt') as input_file:
        for line in [line.strip() for line in input_file.readlines()]:
            vent_lines.append([int(n) for pair in line.split(' -> ') for n in pair.split(',')])

    # fill map
    for line in vent_lines:
        # vertical line
        if (line[0] == line[2]):
            direction = sign(line[3] - line[1])
            for y in range(line[1], line[3] + direction, direction):
                if (line[0], y) not in vent_map:
                    vent_map[(line[0], y)] = 1
                else:
                    vent_map[(line[0], y)] += 1

        # horizontal line
        elif (line[1] == line[3]):
            direction = sign(line[2] - line[0])
            for x in range(line[0], line[2] + direction, direction):
                if (x, line[1]) not in vent_map:
                    vent_map[(x, line[1])] = 1
                else:
                    vent_map[(x, line[1])] += 1

        # diagonal lines
        else:
            x1, y1 = (line[0], line[1]) if line[0] < line[2] else (line[2], line[3])
            x2, y2 = (line[0], line[1]) if line[0] > line[2] else (line[2], line[3])
            gradient = (line[3] - line[1]) / (line[2] - line[0])
            b = y1 - int(x1 * gradient)

            while True:
                if (x1, y1) not in vent_map:
                    vent_map[(x1, y1)] = 1
                else:
                    vent_map[(x1, y1)] += 1

                x1 += 1
                y1 = int(x1 * gradient) + b

                if x1 == x2 and y1 == y2:
                    if (x1, y1) not in vent_map:
                        vent_map[(x1, y1)] = 1
                    else:
                        vent_map[(x1, y1)] += 1
                        
                    break

    # count dangerous coords
    count = len([coord for coord in vent_map if vent_map[coord] > 1])

    print(f'Solution: {count}')

Solution: 18442
